<a href="https://colab.research.google.com/github/Gitsart/2022-MicroControllersBasics/blob/main/Copy_of_ESP32_Multitasking_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#WIFiScan_LEDBlink_DualTASK

import machine
import network
import time
import _thread


LED_PIN = 2  # Most ESP32 dev boards have an onboard LED connected to GPIO2

led_state = False
led_object = None

# --- Task 1: Blink internal LED ---
def blink_led_task():
    global led_object
    led_object = machine.Pin(LED_PIN, machine.Pin.OUT)
    print("LED Blinking Task Started")
    while True:
        try:
            led_object.value(not led_object.value())  # Toggle LED state
            time.sleep(0.5)  # Blink every 0.5 seconds
        except Exception as e:
            print(f"Error in LED task: {e}")
            time.sleep(1) # Wait before retrying


# --- Task 2: Scan Wi-Fi networks ---
def wifi_scan_task():
    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    print("Wi-Fi Scan Task Started")
    while True:
        try:
            print("\nScanning for Wi-Fi networks...")
            networks = wlan.scan() # Perform a Wi-Fi scan
            if networks:
                print("Found Wi-Fi Networks:")
                # Iterate and print network details
                for net in networks:
                    ssid = net[0].decode('utf-8')
                    bssid = ':'.join(['%02x' % b for b in net[1]])
                    channel = net[2]
                    rssi = net[3]
                    authmode = net[4] # Security mode
                    hidden = net[5]

                    auth_modes = {
                        0: "OPEN",
                        1: "WEP",
                        2: "WPA-PSK",
                        3: "WPA2-PSK",
                        4: "WPA/WPA2-PSK",
                        5: "WPA2-ENTERPRISE",
                        6: "WPA3-PSK",
                        7: "WPA2/WPA3-PSK"
                    }
                    auth_display = auth_modes.get(authmode, "UNKNOWN")

                    print(f"  SSID: {ssid}, BSSID: {bssid}, Channel: {channel}, RSSI: {rssi} dBm, Auth: {auth_display}, Hidden: {hidden}")
            else:
                print("No Wi-Fi networks found.")

            time.sleep(10)  # Scan every 10 seconds
        except Exception as e:
            print(f"Error in Wi-Fi scan task: {e}")
            time.sleep(5) # Wait before retrying, maybe network issue

# --- Main execution ---
if __name__ == '__main__':
    print("Starting main program...")

    # Start the LED blinking task in a new thread
    _thread.start_new_thread(blink_led_task, ())

    # Start the Wi-Fi scan task in another new thread
    _thread.start_new_thread(wifi_scan_task, ())

    # The main thread can also do work, or simply sleep to keep the program alive.
    # In this case, we'll just keep the main thread alive indefinitely.
    while True:
        time.sleep(1)

In [ ]:
#WEBPAGE_withTASK

import network
import socket
import machine
import time

# --- Configuration ---
SSID = 'ESP32_Control'      # Wi-Fi network name
PASSWORD = 'micropythonpassword' # Wi-Fi network password (at least 8 characters)
LED_PIN = 2                 # GPIO pin for the onboard LED
USERNAME = 'admin'          # Admin username for authentication
AUTH_PASSWORD = 'password'  # Password for authentication

# --- Global LED object ---
led = machine.Pin(LED_PIN, machine.Pin.OUT)
led_state = False # Initial state: LED is OFF

# --- Simple session management (for demonstration) ---
# In a real application, you'd use more robust session tokens or cookies.
authenticated_clients = {} # Stores client IP addresses that have authenticated

# --- HTML Content ---

# Login Page HTML
LOGIN_PAGE_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        body { font-family: Arial, sans-serif; background-color: #f0f2f5; display: flex; justify-content: center; align-items: center; min-height: 100vh; margin: 0; }
        .container { background-color: #ffffff; padding: 30px; border-radius: 12px; box-shadow: 0 4px 20px rgba(0, 0, 0, 0.1); width: 90%; max-width: 400px; text-align: center; }
        h2 { color: #333; margin-bottom: 25px; }
        .form-group { margin-bottom: 20px; text-align: left; }
        label { display: block; margin-bottom: 8px; font-weight: bold; color: #555; }
        input[type="text"], input[type="password"] { width: calc(100% - 20px); padding: 12px 10px; border: 1px solid #ddd; border-radius: 8px; font-size: 16px; box-sizing: border-box; }
        button { background-color: #007bff; color: white; padding: 12px 25px; border: none; border-radius: 8px; cursor: pointer; font-size: 18px; width: 100%; transition: background-color 0.3s ease; }
        button:hover { background-color: #0056b3; }
        .error-message { color: #dc3545; margin-top: 15px; font-weight: bold; }
    </style>
</head>
<body>
    <div class="container">
        <h2>🔒 ESP32 Login</h2>
        <form action="/login" method="post">
            <div class="form-group">
                <label for="username">Username:</label>
                <input type="text" id="username" name="username" required>
            </div>
            <div class="form-group">
                <label for="password">Password:</label>
                <input type="password" id="password" name="password" required>
            </div>
            <button type="submit">Log In</button>
        </form>
        {error_message}
    </div>
</body>
</html>
"""

# Control Page HTML
CONTROL_PAGE_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>ESP32 LED Control</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        body { font-family: Arial, sans-serif; background-color: #f0f2f5; display: flex; justify-content: center; align-items: center; min-height: 100vh; margin: 0; }
        .container { background-color: #ffffff; padding: 30px; border-radius: 12px; box-shadow: 0 4px 20px rgba(0, 0, 0, 0.1); width: 90%; max-width: 400px; text-align: center; }
        h2 { color: #333; margin-bottom: 25px; }
        .led-status { font-size: 2em; margin-bottom: 30px; }
        .button-group { display: flex; justify-content: center; gap: 20px; }
        .control-button { background-color: #28a745; color: white; padding: 15px 30px; border: none; border-radius: 8px; cursor: pointer; font-size: 20px; transition: background-color 0.3s ease, transform 0.1s ease; }
        .control-button.off { background-color: #dc3545; }
        .control-button:active { transform: scale(0.98); }
        .control-button:hover { opacity: 0.9; }
        .logout-button { margin-top: 30px; background-color: #6c757d; color: white; padding: 10px 20px; border: none; border-radius: 8px; cursor: pointer; font-size: 16px; transition: background-color 0.3s ease; }
        .logout-button:hover { background-color: #5a6268; }
    </style>
    <script>
        function toggleLed() {
            window.location.href = '/toggle_led';
        }
        function logout() {
            window.location.href = '/logout';
        }
    </script>
</head>
<body>
    <div class="container">
        <h2>💡 LED Control</h2>
        <p class="led-status">LED is currently: <strong>{led_status}</strong></p>
        <div class="button-group">
            <button class="control-button {button_class}" onclick="toggleLed()">Turn {button_text}</button>
        </div>
        <button class="logout-button" onclick="logout()">Log Out</button>
    </div>
</body>
</html>
"""

# --- Network Setup ---
def setup_ap():
    ap = network.WLAN(network.AP_IF)
    ap.active(True)
    ap.config(essid=SSID, password=PASSWORD)
    print(f"Access Point '{SSID}' created with IP: {ap.ifconfig()[0]}")
    while ap.active() == False:
        pass
    print("AP setup complete.")
    print("Connect to network:", SSID)
    print("Password:", PASSWORD)
    print("Open http://", ap.ifconfig()[0], "in your browser")


# --- Web Server Functions ---
def handle_request(client_socket, client_address):
    global led_state
    request = client_socket.recv(1024).decode('utf-8')
    # print(f"Request from {client_address[0]}: \n{request}") # Uncomment for debugging

    # Parse request line
    request_lines = request.split('\r\n')
    if not request_lines:
        client_socket.close()
        return

    first_line = request_lines[0]
    method, path, http_version = first_line.split(' ')

    # Get client IP for authentication tracking
    client_ip = client_address[0]

    response_headers = "HTTP/1.1 200 OK\r\nContent-Type: text/html\r\nConnection: close\r\n\r\n"
    response_content = ""
    redirect_header = ""

    # Check authentication status
    is_authenticated = authenticated_clients.get(client_ip, False)

    if path == '/' and method == 'GET':
        # Serve login page
        response_content = LOGIN_PAGE_HTML.format(error_message="")
    elif path == '/login' and method == 'POST':
        # Handle login attempt
        try:
            body = request_lines[-1] # Simple way to get the last line as body
            # Parse form data (very basic, not robust for complex forms)
            username_start = body.find("username=") + len("username=")
            password_start = body.find("password=") + len("password=")

            username_end = body.find("&", username_start)
            password_end = len(body) # Assume password is the last parameter

            username = ""
            password = ""

            if username_start != -1 and username_end != -1:
                username = body[username_start:username_end]
            elif username_start != -1: # if username is the last parameter
                username = body[username_start:]

            if password_start != -1:
                password = body[password_start:password_end]

            # Decode URL-encoded characters (e.g., %40 for @)
            # This is a basic example, a full URL decoder would be more complex
            username = username.replace('%40', '@')
            password = password.replace('%40', '@')

            # print(f"Attempted login - Username: {username}, Password: {password}") # Debug

            if username == USERNAME and password == AUTH_PASSWORD:
                authenticated_clients[client_ip] = True
                redirect_header = "HTTP/1.1 303 See Other\r\nLocation: /control\r\nConnection: close\r\n\r\n"
            else:
                error_msg = "<p class='error-message'>Invalid username or password. Please try again.</p>"
                response_content = LOGIN_PAGE_HTML.format(error_message=error_msg)
        except Exception as e:
            print(f"Error parsing login POST: {e}")
            error_msg = "<p class='error-message'>An error occurred during login. Please try again.</p>"
            response_content = LOGIN_PAGE_HTML.format(error_message=error_msg)

    elif path == '/control' and method == 'GET':
        if is_authenticated:
            # Serve control page
            status_text = "ON" if led_state else "OFF"
            button_text = "OFF" if led_state else "ON"
            button_class = "off" if led_state else ""
            response_content = CONTROL_PAGE_HTML.format(led_status=status_text, button_text=button_text, button_class=button_class)
        else:
            # Not authenticated, redirect to login
            redirect_header = "HTTP/1.1 303 See Other\r\nLocation: /\r\nConnection: close\r\n\r\n"
    elif path == '/toggle_led' and method == 'GET':
        if is_authenticated:
            # Toggle LED and redirect back to control page
            led_state = not led_state
            led.value(led_state)
            print(f"LED is now {'ON' if led_state else 'OFF'}")
            redirect_header = "HTTP/1.1 303 See Other\r\nLocation: /control\r\nConnection: close\r\n\r\n"
        else:
            # Not authenticated, redirect to login
            redirect_header = "HTTP/1.1 303 See Other\r\nLocation: /\r\nConnection: close\r\n\r\n"
    elif path == '/logout' and method == 'GET':
        authenticated_clients[client_ip] = False # Clear authentication for this client
        print(f"Client {client_ip} logged out.")
        redirect_header = "HTTP/1.1 303 See Other\r\nLocation: /\r\nConnection: close\r\n\r\n"
    else:
        # Not found / Bad request
        response_headers = "HTTP/1.1 404 Not Found\r\nContent-Type: text/html\r\nConnection: close\r\n\r\n"
        response_content = "<h1>404 Not Found</h1><p>The requested resource was not found.</p>"

    try:
        if redirect_header:
            client_socket.sendall(redirect_header.encode('utf-8'))
        else:
            client_socket.sendall(response_headers.encode('utf-8') + response_content.encode('utf-8'))
    except Exception as e:
        print(f"Error sending response: {e}")
    finally:
        client_socket.close()


def start_web_server():
    # Create socket
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # Allow reusing address
    s.bind(('', 80)) # Bind to all available interfaces on port 80
    s.listen(5) # Listen for up to 5 incoming connections
    print("Web server listening on port 80")

    while True:
        try:
            conn, addr = s.accept() # Accept a new connection
            handle_request(conn, addr) # Handle the request
        except OSError as e:
            if e.args[0] == 110: # ETIMEDOUT error, common on ESP32
                print("Socket timed out, continuing...")
            else:
                print(f"Socket error: {e}")
        except Exception as e:
            print(f"Error accepting connection or handling request: {e}")

# --- Main execution ---
if __name__ == '__main__':
    print("Starting ESP32 Web Server...")
    setup_ap()
    start_web_server()